In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2023
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [1]:
!pip3 install open-inference-openapi

In [5]:
from open_inference.openapi.client import OpenInferenceClient, InferenceRequest
import httpx
import requests
import json

# Another option is to set this in the project or user environment
MODEL_REGISTRY_URL='https://modelregistry.ml-1c1b0745-332.go01-dem.ylcu-atmi.cloudera.site'

# This comes from the experiment run output above
MODEL_VERSION = 4

JWT = json.load(open("/tmp/jwt"))["access_token"]
headers = {'Authorization': 'Bearer ' + JWT,
           'Content-Type': 'application/json'}

httpx_client = httpx.Client(headers=headers)
r = httpx_client.get(MODEL_REGISTRY_URL+'/api/v1alpha1/deployEndpoint')
models = r.json()
mm = models['models']

my_model = {}
for m in mm:
    if m['name'] == 'fraud-detector-onnx-xgboost':
        my_model = m
        break
MODEL_ID = my_model['id']
MODEL_ID

KeyError: 'models'

In [6]:
models

{'code': 404, 'message': 'path /api/v1alpha1/deployEndpoint was not found'}

### Deploy the Model to CAI Inference Service

In [7]:
JWT = json.load(open("/tmp/jwt"))["access_token"]
headers = {'Authorization': 'Bearer ' + JWT,
           'Content-Type': 'application/json'}

httpx_client = httpx.Client(headers=headers)

In [8]:
CAII_DOMAIN_URL = 'https://ml-2dad9e26-62f.go01-dem.ylcu-atmi.cloudera.site'

deploy_payload = {
 "namespace": "serving-default",
 "name": "Paul Onnx Xgboost",
 "source": {
   "registry_source": {
      "model_id": "3yss-zkky-xrp6-i51n",
      "version": 4
    }
  },
 "resources": {
    "req_cpu": "2",
    "req_memory": "8Gi"
  },
  "api_standard": "oip",
  "has_chat_template": False,
  "metric_format": "triton",
  "task": "INFERENCE",
  "instance_type": "m5.4xlarge",
 "autoscaling": {
    "min_replicas": "1",
    "max_replicas": "100"
  }
}

r = httpx_client.post(CAII_DOMAIN_URL+'/api/v1alpha1/deployEndpoint', json=deploy_payload)
r

<Response [500 Internal Server Error]>

In [4]:
CAII_DOMAIN_URL = 'https://ml-2dad9e26-62f.go01-dem.ylcu-atmi.cloudera.site'
# Wait for the endpoint to be ready
describe_payload = {
    "namespace": "serving-default",
    "name": "XGBoost Fraud Detector"
}

r = httpx_client.post(CAII_DOMAIN_URL+'/api/v1alpha1/describeEndpoint', json=describe_payload)
status = r.json()['status']
response = json.dumps(r.json(), indent=2)
print(response)

# Want target_model_state to get to 'Loaded'
status['target_model_state']

{
  "namespace": "serving-default",
  "name": "XGBoost Fraud Detector",
  "url": "https://ml-2dad9e26-62f.go01-dem.ylcu-atmi.cloudera.site/namespaces/serving-default/endpoints/xgboost-fraud-detector/v2/models/3yss-zkky-xrp6-i51n/infer",
  "conditions": [
    {
      "type": "IngressReady",
      "status": "True",
      "severity": "",
      "last_transition_time": "1742515403",
      "reason": "",
      "message": ""
    },
    {
      "type": "LatestDeploymentReady",
      "status": "True",
      "severity": "Info",
      "last_transition_time": "1742515403",
      "reason": "",
      "message": ""
    },
    {
      "type": "PredictorConfigurationReady",
      "status": "True",
      "severity": "Info",
      "last_transition_time": "1742515403",
      "reason": "",
      "message": ""
    },
    {
      "type": "PredictorReady",
      "status": "True",
      "severity": "",
      "last_transition_time": "1742515403",
      "reason": "",
      "message": ""
    },
    {
      "type":

'Loaded'

In [38]:
r

<Response [404 Not Found]>